Import libraries

In [1]:
import sys
import rospy
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import time
import math
import jupyros as jr
import numpy as np
import ipywidgets as widgets
import select
import matplotlib.pyplot as plt
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2022.msg import Position_velocity
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
%matplotlib widget

Defining methods to handle the click on the button

In [2]:
#This function is called when the reach goal button is clicked
def reach_goal_button_clicked(_):
    global pos_x, pos_y
    #Retrive the x and y value from the input fields
    pos_x = goal_x_input.value
    pos_y = goal_y_input.value
    #Create a goal message
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = pos_x
    goal.target_pose.pose.position.y = pos_y
    #Send the goal to the action server
    user.send_goal(goal)
    vis.goal_callback(goal)

#This function is called when the cancel button is clicked
def cancel_goal_button_clicked(_):
    #It checks if there is a goal already setted, if it is then cancel the goal
    if user.get_state() == actionlib.GoalStatus.ACTIVE:
        user.cancel_goal()
        print("Goal cancelled!")
    #Otherwise it send a message that the is no active goal to cancel
    else:
        print("No active goal to cancel")

Defining the widget for the current position

In [3]:
#Defining two widgets to display the current position of the robot (x, y)
current_pos_x=widgets.FloatText(description='Current position x:', disabled = True)
current_pos_y=widgets.FloatText(description='Current position y:', disabled = True)

Class visualizer for visualizing the graphic of robot position and statistics

In [4]:
global user
#Class for visualizing data on the plot
class Visualiser:
    def __init__(self):
        self.fig, (self.ax, self.ax2) = plt.subplots(2, 1, figsize=(8, 10))
        
        self.ln, = self.ax.plot([], [], 'ro')
        
        
        self.x_data, self.y_data = [], []
        self.goal_x_data, self.goal_y_data = [], []

        self.reached_goals = 0
        self.cancelled_goals = 0
        self.current_goal_reached = False
        self.goal_canceled = False
    
    #This function is used to initialize the graph by setting the x and y limits of the axes.
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax2.set_ylim(0, 10)
        self.ax2.set_yticks(np.arange(0,11,1))
        self.ax2.grid(axis = 'y', color = 'grey', linestyle = '-', alpha = 0.5)
        return self.ln
    
    #This function stores the position data for odom message and updates the current position widgets.
    def odom_callback(self, odom_msg):
        self.y_data.append(odom_msg.pose.pose.position.y)
        self.x_data.append(odom_msg.pose.pose.position.x)
        current_pos_x.value = round(odom_msg.pose.pose.position.x, 2)
        current_pos_y.value = round(odom_msg.pose.pose.position.y, 2)
    
    #This function stores the goal position data
    def goal_callback(self, goal_msg):
        self.goal_x_data.append(goal_msg.target_pose.pose.position.x)
        self.goal_y_data.append(goal_msg.target_pose.pose.position.y)
    
    #This function updates the graph based on the current status of the goal.
    #It plots the robot's trajectory, marks reached and canceled goals, and updates the goal statistics.
    def update_plot(self, frame):
        status = user.get_state()
        self.ln.set_data(self.x_data, self.y_data)
        # check if the goal is reached
        if status==3 and not(self.current_goal_reached):
            self.ax.plot(self.goal_x_data[-1:], self.goal_y_data[-1:], 'go', label='Reached goals') 
            self.reached_goals += 1
            self.ax2.bar(['Reached Goals', 'Cancelled Goals'], [self.reached_goals, self.cancelled_goals], color = 'orange')
            self.current_goal_reached=True
            
        # check if the goal has been cancelled
        elif status==2 and not(self.goal_canceled):
            self.ax.plot(self.goal_x_data[-1:], self.goal_y_data[-1:], 'yo', label='Cancelled goals')
            self.cancelled_goals += 1
            self.ax2.bar(['Reached Goals', 'Cancelled Goals'], [self.reached_goals, self.cancelled_goals], color = 'orange')
            self.goal_canceled=True
        
        elif (not(status==3) and self.current_goal_reached):
            self.current_goal_reached=False 
            
        elif (not(status==2) and self.goal_canceled):
            self.goal_canceled=False
            
        # check if the robot is going to the current goal
        elif (not(status==3) and not(status==2)):
            self.ax.plot(self.goal_x_data[-1:], self.goal_y_data[-1:], 'bo', label='Current goals')
            
        self.ax2.bar(['Reached Goals', 'Cancelled Goals'], [self.reached_goals, self.cancelled_goals], color = 'orange')
        return self.ln

Initialize the node and defining the widget

In [5]:
#INitialize the node
rospy.init_node('user')

#create an instance of the Visualiser class
vis = Visualiser()

#set titles for the graph
vis.ax.set_title('Robot trajectory')
vis.ax2.set_title('Number of goals reached and deleted')

#Create publishers and subscribers
pub = rospy.Publisher("/position_velocity", Position_velocity, queue_size=1)
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)

user = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
user.wait_for_server()

#Define the widgets for user interface
goal_x_input = widgets.FloatText(description='Target X:')
goal_y_input = widgets.FloatText(description='Target Y:')
reach_goal_button = widgets.Button(description='Reach Goal')
cancel_goal_button = widgets.Button(description='Cancel Goal')
output_distance = widgets.FloatText(description='Closest obstacle distance:', value=0.0)

reach_goal_button.on_click(reach_goal_button_clicked)
cancel_goal_button.on_click(cancel_goal_button_clicked)

menu_box = widgets.VBox([
    widgets.Label('Welcome to the Main Menu'),
    goal_x_input,
    goal_y_input,
    reach_goal_button,
    cancel_goal_button,
    output_distance,
    current_pos_x,
    current_pos_y,
])

display(menu_box)

#This line creates an animation using the update_plot function of the Visualiser class and displays the graphic
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Defining callback to identify the closest obstacle

In [6]:
def laser_scan_callback(scan_msg):
    closest_obstacle_distance = scan_msg.range_max
    for range_val in scan_msg.ranges:
        if range_val < closest_obstacle_distance:
            closest_obstacle_distance = range_val
    output_distance.value = round(closest_obstacle_distance, 2)

# Effettua la subscription per ricevere i messaggi del laser scanner
jr.subscribe('/scan', LaserScan, laser_scan_callback)